# <center> R1.13 introduction à Python <br> TP4 - Nim, Wythoff et IA </center>
<center> 2023/2024 - Thibault Godin </center>
<center> IUT de Vannes, BUT Informatique </center>

***

Dans ce TP, on va voir une méthode d'intelligence artificielle classique, **l'apprentissage par renforcement**


On va présenter cette notion à l'aide d'une activité proposée par des collègues de la (Maison des Mathématiques et de l'Informatique)[https://mmi-lyon.fr/] (MMI) et du réseau Informatique Débranchée sur le jeu de _Nim_, puis on essaiera de l'appliquer au jeu de Withoff vu en JAVA en R1.01



- https://mmi-lyon.fr/?site_ressource_peda=jeu-de-nim-et-ia-avec-python
- https://mmi-lyon.fr/?site_ressource_peda=jeu-de-nim-et-ia

In [2]:
import random
import time

import numpy as np


import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 12)


# Jeu de Nim


## Mise en place


Le jeu de Nim se joue à 2 joueurs selon les règles suivantes : 
- On commence la partie avec un tas de $n$ allumettes
- Les deux joueurs jouent alternativement
- Seulement 1, 2, ou 3 allumette(s) peuvent être retirée(s) par un joueur à chaque tour
- La personne qui tire la dernière alumettre a gagné (version normale, on peut aussi dire "la personne qui ne peut plus jouer a perdu")

>**_question 1_** <br>
jouer (en silence) une ou deux parties avec un voisin

>**_question 2_**<br> 
lire (en diagonale) à quoi ressemblent les [stratégies gagnantes](https://interstices.info/jeux-de-nim/) de ce jeu 

>**_question 3_**<br> 
visualiser et comprendre l'apprentissage par renforcement proposé pour le jeu de Nim sur https://projet.liris.cnrs.fr/~mam/machine/




--------------------------



## Niammi : Nim Artificial Intelligence Maison des Mathématiques et de l'Informatique

Voici un programme Python (implémenté par Olivier Druet de la MMI) permettant de jouer au jeu de Nim contre un ordinateur _qui apprend au fur et à mieux jouer_. 

>**_question 4_**<br> 
lire et comprendre

In [7]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Jan 21 10:32:50 2022

@author: druet
"""
import numpy as np

import random
import time


################### le message de bienvenue et d'explication ##################

def welcome_message():
    print("""
----------------------------------------------------------------------------
                        Règles du jeu de Nim
----------------------------------------------------------------------------
Règles : 1 - L'utilisateur et l'ordinateur jouent à tour de rôle.
         2 - L'ordinateur commence.
         3 - Seulement 1 ou 2 bâtons peuvent être retirés à chaque coup.
         4 - Celui qui tire le dernier bâton a gagné.
         
----------------------------------------------------------------------------
                           Apprentissage
----------------------------------------------------------------------------                        
Suivant qu'il gagne ou perd, l'ordinateur reçoit une récompense ou une punition.
----------------------------------------------------------------------------
""")

#### Nous faisons commencer l'ordinateur parce que le premier joueur a une 
#### stratégie gagnante. Si le joueur commence et qu'il joue bien, la machine 
#### perdra quoiqu'elle fasse et n'apprendra rien. 

###############################################################################


################## définition et impression de la position de jeu #############

def printboard(n):
    board=[]
    for _ in range(n):
        board.append("/")
    print("\n----------------------------------------------------------------------------")
    print("      ",*board, sep="   ")
    print("----------------------------------------------------------------------------\n")
    print("Il reste " + str(n) + " allumettes.")
    
###############################################################################


################### initialisation de la machine ##############################

nombre_allumettes=8 ### vous pouvez changer le nombre d'allumettes de départ
board = [] ### le plateau de jeu
boulesjaunes = [] ### correspond au nombre de boules jaunes dans la case
boulesrouges = [] ### correspond au nombre de boules rouges dans la case
tirage = [] ### correspondra au tirage dans une partie
for _ in range(nombre_allumettes):
    boulesjaunes.append(2)
    boulesrouges.append(2)
    tirage.append(0)
#### Attention Python commence à 0 #####
boulesrouges[0]=0 #### il ne faut pas mettre de boules rouges dans la case 0, coup interdit


###############################################################################


#################### programme principal : jeu + renforcement #################
welcome_message()
uneautrepartie=True
compteur_partie=0
time.sleep(1)
print("La probabilité de gain de l'ordinateur à la première partie si le joueur joue optimalement est de 12,5 %")
time.sleep(1)
while uneautrepartie :
    player = "CMP"
    allumettes=nombre_allumettes
    tirage=[0,0,0,0,0,0,0,0]
    while allumettes>0:
        printboard(allumettes)  ### imprime la position de jeu
        time.sleep(1)
        if player=='CMP': ### c'est à l'ordinateur de jouer
            print("\nL'ordinateur choisit de retirer...")
            time.sleep(1)
            somme=boulesjaunes[allumettes-1]+boulesrouges[allumettes-1]
            boulehasard=random.randint(1,somme)##permet de tirer jaune ou rouge
            if boulehasard <= boulesjaunes[allumettes-1]: ##la machine a tiré jaune, i.e. elle enlève une allumette
                tirage[allumettes-1]=1
                allumettes=allumettes-1
                print("1 allumette.")
            else:
                tirage[allumettes-1]=2
                allumettes=allumettes-2
                print("2 allumettes.")
            if allumettes==0:
                winner='CMP'
            else:
                player='USER'  
        else: ### c'est au joueur de jouer
#### on demande au joueur ce qu'il souhaite jouer avec vérification que c'est un coup légal
            coup_joueur=0
            print("\n--À vous de jouer !--")
            while coup_joueur not in range(1, 3) or coup_joueur>allumettes:
                try:
                    coup_joueur = int(input("\nQuel est votre choix ?"))
                    if coup_joueur == 0:
                        print("\nVous devez enlever au moins une allumette !")
                    elif coup_joueur not in range(1, 3) or coup_joueur>allumettes:
                        print("\nVous ne pouvez pas enlever autant d'allumettes !")
                        coup_joueur = int(input("\nQuel est votre choix ?"))
                except Exception as e:
                    print("\nCela ne semble pas une réponse valide.\nError: " + str(e) + "\nRecommencez !")
##### fin du choix du joueur
            allumettes=allumettes-coup_joueur
            if allumettes==0:
                winner='USER'
            else:
                player='CMP' 
    compteur_partie+=1
##### fin de la partie ######
##### annonce des résultats #####
    if winner=='CMP':
        print("\n----------------------------------------------------------------------------")
        print("L'ordinateur a gagné, nous allons le récompenser.")
        print("----------------------------------------------------------------------------\n")
    else:
        print("\n----------------------------------------------------------------------------")
        print("Bravo ! Vous avez gagné ! Nous allons punir l'ordinateur.")
        print("----------------------------------------------------------------------------\n")
##### Apprentissage : récompense ou punition de l'ordinateur#####        
    if winner=='CMP': ###récompense
        for i in range(nombre_allumettes):
            if tirage[i]==1:
                boulesjaunes[i]=boulesjaunes[i]+1
            if tirage[i]==2:
                boulesrouges[i]=boulesrouges[i]+1
    else: ###punition
        for i in range(nombre_allumettes):
            if tirage[i]==1:
                boulesjaunes[i]=boulesjaunes[i]-1
            if tirage[i]==2:
                boulesrouges[i]=boulesrouges[i]-1
####### fin de la récompense ou de la punition #################
####### réinitialisation des verres vides ######################
    for i in range(nombre_allumettes):
        if (boulesjaunes[i]==0) and (boulesrouges[i]==0):
            boulesjaunes[i]=2
            boulesrouges[i]=2
######## impression de l'état des verres #######################
    time.sleep(1)
    for i in range(nombre_allumettes):
        print("Dans le verre " + str(i+1) +", il y a " + str(boulesjaunes[i]) + " boules jaunes et " + str(boulesrouges[i]) + " boules rouges.")
################################################################
############ calcul de la probabilité de gagner ################
    time.sleep(1)
    if (boulesjaunes[3]/(boulesjaunes[3]+boulesrouges[3]))>(boulesrouges[4]/(boulesjaunes[4]+boulesrouges[4])):
        proba=(boulesrouges[7]/(boulesjaunes[7]+boulesrouges[7]))*(boulesrouges[4]/(boulesjaunes[4]+boulesrouges[4]))*(boulesrouges[1]/(boulesjaunes[1]+boulesrouges[1]))
    else:
        proba=(boulesrouges[7]/(boulesjaunes[7]+boulesrouges[7]))*(boulesjaunes[3]/(boulesjaunes[3]+boulesrouges[3]))*(boulesrouges[1]/(boulesjaunes[1]+boulesrouges[1]))
    print("\n----------------------------------------------------------------------------")
    print("Vous avez joué "+ str(compteur_partie) +" parties.")
    print("----------------------------------------------------------------------------\n")
    print("\n----------------------------------------------------------------------------")
    print("La probabilité de gain de l'ordinateur à la prochaine partie si le joueur joue optimalement (en connaissant l'état des verres :-)) est de " + str(round(proba*100,2)) + "%")
    print("----------------------------------------------------------------------------\n")
################################################################
############ On continue ? ################    
    test=True
    while test:
        another_go = input("\nVoulez-vous rejouer ?[O/N]: ")
        if another_go in ("o","O"):
            uneautrepartie=True
            test=False
        elif another_go in ("n","N"):
            uneautrepartie=False
            test=False
        else:
            print("\nChoix invalide. Recommencez !")    
############################################            
            




 



----------------------------------------------------------------------------
                        Règles du jeu de Nim
----------------------------------------------------------------------------
Règles : 1 - L'utilisateur et l'ordinateur jouent à tour de rôle.
         2 - L'ordinateur commence.
         3 - Seulement 1 ou 2 bâtons peuvent être retirés à chaque coup.
         4 - Celui qui tire le dernier bâton a gagné.
         
----------------------------------------------------------------------------
                           Apprentissage
----------------------------------------------------------------------------                        
Suivant qu'il gagne ou perd, l'ordinateur reçoit une récompense ou une punition.
----------------------------------------------------------------------------

La probabilité de gain de l'ordinateur à la première partie si le joueur joue optimalement est de 12,5 %

----------------------------------------------------------------------------


# Wythoff


plan conseillé : 

1. affichage du plateau
2. alternance des joueurs et propositions des coups admissibles
3. condition de victoire
4. jeu humain v. humain
5. stockage de stratégie
6. jeu humain v. ordinateur (aléatoire)
8. règle de l'apprentissage de l'ordinateur (en cas de victoire ou de défaite
9. entrainement de l'ordinateur contre l'humain
10. entrainement de l'ordinateur contre le hasard
11. entrainement de l'ordinateur contre l'ordinateur

Vous pouvez bien sûr recycler du code Java de R1.01, ainsi que le code de Nim proposé plus haut (essayez d'être aussi clair dans votre code).

Le jeu de Wythoff étant plus complexe que celui de Nim, on fera des fonctions !

Pensez à documenter, commenter et illustrer votre TP.

**bonus** Une fois la machine entrainée, retrouver les positions gagnante à l'aide de la stratégie obtenue ; ou analyser la qualité de l'entrainement en fonction des paramètres de renforcement.

In [3]:
import random
import time
import numpy as np

def creerPlateau(n):
    plateauJeu = np.full((n, n), 0)  # Crée un plateau de jeu n x n avec des zéros
    return plateauJeu

def affichePlateau(plateauJeu):
    print(plateauJeu)  # Affiche le plateau de jeu

def joueurOuIA():
    while True:
        try:
            choix = int(input("\nChoisissez si vous jouez contre un joueur(1) ou contre une IA(2) ?\n"))
            if choix == 1:
                joueur = str(input("Nom du joueur 2 : "))
                break
            elif choix == 2:
                joueur = 'IA'
                print("Vous jouez contre l'IA")
                break
        except ValueError:
            print("Veuillez entrer un nombre valide. \n")
    return joueur

# La fonction demarrage initialise le jeu
def demarrage():
    global joueur1, joueur2, joueurActuelle, points_joueur1, points_joueur2
    joueur1 = str(input("Nom du joueur 1 : "))
    joueur2 = joueurOuIA()

    points_joueur1 = 0
    points_joueur2 = 0

    while True:
        longueur = int(input("\nChoisissez une longueur ?\n"))
        plateauJeu = creerPlateau(longueur)
        pionDepart(plateauJeu)
        affichePlateau(plateauJeu)
        print(joueur2)
        if joueur2 == 'IA':
            jeuAvecIA(plateauJeu)
        else:
            jeu(plateauJeu)

        if victoire(plateauJeu, joueur1):
            points_joueur1 += 1
        elif victoire(plateauJeu, joueur2):
            points_joueur2 += 1

        print("Scores -", joueur1, ":", points_joueur1, "|", joueur2, ":", points_joueur2)

        if not partie_continue():
            break

    # Afficher les scores à la fin de la partie
    print("\nScores finaux -", joueur1, ":", points_joueur1, "|", joueur2, ":", points_joueur2)

# La fonction pionDepart place le pion du joueur au début du jeu
def pionDepart(plateauJeu):
    global positionX
    global positionY
    positionRandom1 = random.randint(0, 1)
    positionRandom2 = random.randint(1, plateauJeu.shape[0] - 2)

    if positionRandom1 == 0:
        positionX = 0
        positionY = positionRandom2
        plateauJeu[positionX][positionY] = 1
    elif positionRandom1 == 1:
        positionX = positionRandom2
        positionY = plateauJeu.shape[1] - 1
        plateauJeu[positionX][positionY] = 1

# La fonction change_joueur change le joueur actuel
def change_joueur(USER):
    global joueur1, joueur2, joueurActuelle

    if joueurActuelle == joueur2:
        joueurActuelle = joueur1
    else:
        joueurActuelle = joueur2
    return joueurActuelle

# La fonction estDansPlateau vérifie si le déplacement est dans les limites du plateau
def estDansPlateau(plateauJeu, nbDeplacement, deplacement):
    estDedans = False
    if nbDeplacement > 0:
        if deplacement == 3:
            if positionX + nbDeplacement < len(plateauJeu):
                estDedans = True
        elif deplacement == 1:
            if positionY - nbDeplacement >= 0:
                estDedans = True
        elif deplacement == 2:
            if (
                positionX + nbDeplacement < len(plateauJeu)
                and positionY - nbDeplacement >= 0
            ):
                estDedans = True
    return estDedans

# La fonction deplacer_pion permet au joueur de déplacer son pion
def deplacer_pion(plateauJeu):
    global positionX, positionY
    print("1. Gauche | 2. Diagonal | 3. Bas")

    while True:
        try:
            deplacement = int(input("Dans quel sens allez-vous ?"))
            if deplacement == 1 or deplacement == 2 or deplacement == 3:
                break
        except ValueError:
            print("Veuillez entrer un nombre valide. \n")

    while True:
        try:
            nbDeplacement = int(input("Veuillez indiquer le nombre de déplacements : "))
            if estDansPlateau(plateauJeu, nbDeplacement, deplacement):
                break
            else:
                print("Veuillez entrer un nombre qui reste sur le plateau \n")
        except ValueError:
            print("Veuillez entrer un nombre valide.")

    plateauJeu[positionX][positionY] = 0

    if deplacement == 1:  # deplacement vers la gauche
        positionY -= nbDeplacement
    if deplacement == 2:  # deplacement en diagonal
        positionY -= nbDeplacement
        positionX += nbDeplacement
    if deplacement == 3:  # deplacement vers le bas
        positionX += nbDeplacement

    plateauJeu[positionX][positionY] = 1

# La fonction deplacer_pion_IA_aleatoire permet à l'IA de déplacer son pion de manière aléatoire
def deplacer_pion_IA_aleatoire(plateauJeu):
    global positionX, positionY
    while True:
        mouvementRandom = random.randint(1, 3)

        if mouvementRandom == 1:
            mouvement = "gauche"
            nbCaseRandom = random.randint(1, positionY + 1)
        elif mouvementRandom == 2:
            mouvement = "bas"
            nbCaseRandom = random.randint(1, len(plateauJeu) - positionX)
        else:
            mouvement = "diagonal"
            nbCaseRandom = random.randint(1, min(positionY, positionX) + 1)

        if estDansPlateau(plateauJeu, nbCaseRandom, mouvementRandom):
            break

    plateauJeu[positionX][positionY] = 0

    if mouvementRandom == 1:  # deplacement vers la gauche
        positionY -= nbCaseRandom
    if mouvementRandom == 2:  # deplacement en diagonal
        positionY -= nbCaseRandom
        positionX += nbCaseRandom
    if mouvementRandom == 3:  # deplacement vers le bas
        positionX += nbCaseRandom

    plateauJeu[positionX][positionY] = 1

# La fonction jeu gère le déroulement du jeu entre deux joueurs humains
def jeu(plateauJeu):
    global joueur1, joueur2, joueurActuelle
    while not victoire(plateauJeu, joueurActuelle):
        joueurActuelle = change_joueur(joueurActuelle)
        print("\nC'est au tour de : ", joueurActuelle)
        deplacer_pion(plateauJeu)
        affichePlateau(plateauJeu)

# La fonction jeuAvecIA gère le déroulement du jeu contre l'IA
def jeuAvecIA(plateauJeu):
    global joueur1, joueur2, joueurActuelle
    joueurActuelle = joueur1  # Ajout de cette ligne
    while True:
        joueurActuelle = change_joueur(joueurActuelle)
        print("\nC'est au tour de : ", joueurActuelle)
        if joueurActuelle == joueur1:
            deplacer_pion(plateauJeu)
        else:
            deplacer_pion_IA_aleatoire(plateauJeu)

        affichePlateau(plateauJeu)

        # Stocker le résultat de victoire dans une variable
        est_victoire = victoire(plateauJeu, joueurActuelle)

        if est_victoire:
            print(joueurActuelle, " à gagné !")
            break

# La fonction victoire vérifie si le joueur actuel a gagné
def victoire(plateauJeu, joueurActuelle):
    estFini = False
    if plateauJeu[len(plateauJeu) - 1][0] == 1:
        estFini = True
        return True
    return estFini

# La fonction partie_continue demande au joueur s'il veut continuer à jouer
def partie_continue():
    while True:
        choix = input("Voulez-vous continuer la partie ? (o/n) ")
        if choix.lower() == 'o':
            return True
        elif choix.lower() == 'n':
            return False
        else:
            print("Veuillez entrer 'o' pour continuer ou 'n' pour arrêter.")

# La fonction jouer_partie coordonne le déroulement de la partie
def jouer_partie():
    demarrage()
    while partie_continue():
        demarrage()

# Lancer le programme
jouer_partie()

Vous jouez contre l'IA
[[0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
IA

C'est au tour de :  IA
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]

C'est au tour de :  Omer
1. Gauche | 2. Diagonal | 3. Bas
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]

C'est au tour de :  IA
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0